#  問題2

##  (1)

###  counter_t の定義

In [ ]:
%%writefile counter.h


#include <pthread.h>

typedef struct {
    long v;
    pthread_mutex_t m;
} counter_t;

###  counter_init, counter_dec の定義

In [ ]:
%%writefile counter.c

#include <pthread.h>
#include "counter.h"

void counter_init(counter_t *c, long v) {
    c->v = v;
    ptrhead_mutex_init(c->m, NULL);
}

void counter_dec(counter_t * c, long x) {
    pthread_mutex_lock(&m);
    if (c->v < x) {
        return 0;
    } else {
        c->v -= x;
        return 1;
    }
    pthread_mutex_unlock(&m);
}

##  (2)

###  counter_t の定義

In [ ]:
%%writefile counter_cas.h

typedef struct {
    long v;
} counter_t;

###  counter_init, counter_dec の定義

In [ ]:
%%writefile counter_cas.c

#include "counter_cas.h"

void counter_init(counter_t *c, long v) {
    c->v = v;
}

void counter_dec(counter_t * c, long x) {
    if (x == 0) return 1; // check
    if (c->v < x) { // if c_v < x, then c_v - x < 0, (c_v - x) / x == -1
        // dunno if operations within this is considered atomic
        // (probably not) but lecture note had this example so i'm
        // going with it wheeeeeeee
        long temp = (c->v - x) / x;
        return __sync_bool_compare_and_swap(&long, -1, c->x - x); // rethink this it's probably wrong after u added the long temp = ...
    } 
}

##  (3)
条件1を満たしているか(どちらかを残す)?

1: YES
2: YES
3: NO

NOならばそれを示す実行系列

C[M] = (3, 2), M = 2

customer 1: p,q,x,y = 0,1,3,3
customer 2: p,q,x,y = 0,1,1,1

1に対しxがdecされ0になって、
2に対しxが買いえない状態であり購入が中止される
1が商品1が買えないのでそれも中止
しかしもともと2の購入は成立する -> 3 no

##  (4)
条件1を満たしているか(どちらかを残す)?

1: NO
2: YES
3: YES

NOならばそれを示す実行系列

C[M] = (3, 2, 3), M = 3

customer 1: p,q,x,y = 0,1,3,3
customer 2: p,q,x,y = 1,2,1,1
customer 3: p,q,x,y = 2,0,1,1

1 no: deadlockが起こる：
1 lock p=0
2 lock p=1
3 lock p=2
全部ロックされているため実行が進まない


##  (5)
条件1を満たしているか(どちらかを残す)?

all YES

NOならばそれを示す実行系列

YOUR ANSWER HERE


##  (6)

概略

YOUR ANSWER HERE

In [ ]:

%%writefile buy2.c
#include <assert.h>
#include "counter.h"
#include "buy2.h"

int buy2(counter_t * C, long M, long p, long q, long x, long y) {
  assert(0 <= p && p < M && 0 <= q && q < M);
  
}


テスト: 以下の, `CHANGE_IT_TO_THE_RIGHT_PASSWORD` の部分を正しいパスワード
(問題文に記載)に書き換えて実行せよ. 無事実行できたら, `check_buy2.c`, `buy2.h` というファイルができる. 

In [ ]:
password=CHANGE_IT_TO_THE_RIGHT_PASSWORD
openssl enc -d -aes-256-cbc -pbkdf2 -iter 100000 -in check_buy2.c.encrypted -out check_buy2.c -k ${password}
openssl enc -d -aes-256-cbc -pbkdf2 -iter 100000 -in buy2.h.encrypted -out buy2.h -k ${password}
ls -l

できたら以下でコンパイル, 実行せよ

In [ ]:
gcc -o check_buy2 -Wall -Wextra -O3 -fopenmp counter.c buy2.c check_buy2.c

In [ ]:
OMP_NUM_THREADS=10 timeout 10 ./check_buy2 100 1000000 91807290723